<a href="https://colab.research.google.com/github/duper203/RAG_Techniques_with_upstage/blob/main/upstage/08_contextual_chunk_headers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contextual Chunk Headers (CCH)



## Key Components

### Contextual chunk headers

The idea here is to add in higher-level context to the chunk by prepending a chunk header. This chunk header could be as simple as just the document title, or it could use a combination of document title, a concise document summary, and the full hierarchy of section and sub-section titles.




## Method Details

1. Context generation

2. Embed chunks with chunk headers

3. Add chunk headers to search results

In [ ]:
! pip3 install -qU langchain-upstage langchain langchain-community pypdf faiss-cpu

In [ ]:
import os
from google.colab import userdata

os.environ["UPSTAGE_API_KEY"] = userdata.get("UPSTAGE_API_KEY")

## Define document(s) path

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_into_chunks(text: str, chunk_size: int = 800) -> list[str]:
    """
    Split a given text into chunks of specified size using RecursiveCharacterTextSplitter.

    Args:
        text (str): The input text to be split into chunks.
        chunk_size (int, optional): The maximum size of each chunk. Defaults to 800.

    Returns:
        list[str]: A list of text chunks.

    Example:
        >>> text = "This is a sample text to be split into chunks."
        >>> chunks = split_into_chunks(text, chunk_size=10)
        >>> print(chunks)
        ['This is a', 'sample', 'text to', 'be split', 'into', 'chunks.']
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=0,
        length_function=len
    )
    documents = text_splitter.create_documents([text])
    return [document.page_content for document in documents]

# File path for the input document
FILE_PATH = "data/nike_2023_annual_report.txt"

# Read the document and split it into chunks
with open(FILE_PATH, "r") as file:
    document_text = file.read()

chunks = split_into_chunks(document_text, chunk_size=800)

In [ ]:
from openai import OpenAI
from langchain.prompts import PromptTemplate
import tiktoken
import os
# Constants
DOCUMENT_TITLE_PROMPT = """
INSTRUCTIONS
What is the title of the following document?

Your response MUST be the title of the document, and nothing else. DO NOT respond with anything else.

{document_title_guidance}

{truncation_message}

DOCUMENT
{document_text}
""".strip()

TRUNCATION_MESSAGE = """
Also note that the document text provided below is just the first ~{num_words} words of the document. That should be plenty for this task. Your response should still pertain to the entire document, not just the text provided below.
""".strip()

MAX_CONTENT_TOKENS = 1700
MODEL_NAME = "solar-pro"
# TOKEN_ENCODER = tiktoken.get_encoding()

def make_llm_call(chat_messages: list[dict]) -> str:
    """
    Make an API call to the OpenAI language model.

    Args:
        chat_messages (list[dict]): A list of message dictionaries for the chat completion.

    Returns:
        str: The generated response from the language model.
    """
    client = OpenAI(api_key=os.environ["UPSTAGE_API_KEY"], base_url="https://api.upstage.ai/v1/solar")
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=chat_messages,
        max_tokens=MAX_CONTENT_TOKENS,
        temperature=0.2,
    )
    return response.choices[0].message.content.strip()

def truncate_content(content: str, max_tokens: int) -> tuple[str, int]:
    """
    Truncate the content to a specified maximum number of tokens.

    Args:
        content (str): The input text to be truncated.
        max_tokens (int): The maximum number of tokens to keep.

    Returns:
        tuple[str, int]: A tuple containing the truncated content and the number of tokens.
    """
    encoder = tiktoken.get_encoding("o200k_base")

    # Encode the content into tokens
    tokens = encoder.encode(content)

    # Truncate the tokens
    truncated_tokens = tokens[:max_tokens]

    # Decode the truncated tokens back to text
    truncated_content = encoder.decode(truncated_tokens)

    return truncated_content, len(truncated_tokens)

def get_document_title(document_text: str, document_title_guidance: str = "") -> str:
    """
    Extract the title of a document using a language model.

    Args:
        document_text (str): The text of the document.
        document_title_guidance (str, optional): Additional guidance for title extraction. Defaults to "".

    Returns:
        str: The extracted document title.
    """
    # Truncate the content if it's too long
    document_text, num_tokens = truncate_content(document_text, MAX_CONTENT_TOKENS)
    truncation_message = TRUNCATION_MESSAGE.format(num_words=3000) if num_tokens >= MAX_CONTENT_TOKENS else ""

    # Prepare the prompt for title extraction
    prompt = DOCUMENT_TITLE_PROMPT.format(
        document_title_guidance=document_title_guidance,
        document_text=document_text,
        truncation_message=truncation_message
    )
    chat_messages = [{"role": "user", "content": prompt}]

    return make_llm_call(chat_messages)

# Example usage
if __name__ == "__main__":
    # Assuming document_text is defined elsewhere
    document_title = get_document_title(document_text)
    print(f"Document Title: {document_title}")

Document Title: ANNUAL REPORT ON FORM 10-K


In [ ]:
pip install cohere

In [17]:
import os
from google.colab import userdata

os.environ["CO_API_KEY"] = userdata.get('CO_API_KEY') # Cohere API key

In [21]:
from typing import List
from langchain.prompts import PromptTemplate
import cohere

def rerank_documents(query: str, chunks: List[str]) -> List[float]:
    """
    Use Cohere Rerank API to rerank the search results.

    Args:
        query (str): The search query.
        chunks (List[str]): List of document chunks to be reranked.

    Returns:
        List[float]: List of similarity scores for each chunk, in the original order.
    """
    MODEL = "rerank-english-v3.0"
    client = cohere.Client(api_key=os.environ["CO_API_KEY"])

    reranked_results = client.rerank(model=MODEL, query=query, documents=chunks)
    results = reranked_results.results
    reranked_indices = [result.index for result in results]
    reranked_similarity_scores = [result.relevance_score for result in results]

    # Convert back to order of original documents
    similarity_scores = [0] * len(chunks)
    for i, index in enumerate(reranked_indices):
        similarity_scores[index] = reranked_similarity_scores[i]

    return similarity_scores

def compare_chunk_similarities(chunk_index: int, chunks: List[str], document_title: str, query: str) -> None:
    """
    Compare similarity scores for a chunk with and without a contextual header.

    Args:
        chunk_index (int): Index of the chunk to inspect.
        chunks (List[str]): List of all document chunks.
        document_title (str): Title of the document.
        query (str): The search query to use for comparison.

    Prints:
        Chunk header, chunk text, query, and similarity scores with and without the header.
    """
    chunk_text = chunks[chunk_index]
    chunk_wo_header = chunk_text
    chunk_w_header = f"Document Title: {document_title}\n\n{chunk_text}"

    similarity_scores = rerank_documents(query, [chunk_wo_header, chunk_w_header])

    print(f"\nChunk header:\nDocument Title: {document_title}")
    print(f"\nChunk text:\n{chunk_text}")
    print(f"\nQuery: {query}")
    print(f"\nSimilarity without contextual chunk header: {similarity_scores[0]:.4f}")
    print(f"Similarity with contextual chunk header: {similarity_scores[1]:.4f}")

# Notebook cell for execution
# Assuming chunks and document_title are defined in previous cells
CHUNK_INDEX_TO_INSPECT = 86
QUERY = "Nike climate change impact"

compare_chunk_similarities(CHUNK_INDEX_TO_INSPECT, chunks, document_title, QUERY)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml

Chunk header:
Document Title: ANNUAL REPORT ON FORM 10-K

Chunk text:
Given the broad and global scope of our operations, we are particularly vulnerable to the physical risks of climate change, such 
as shifts in weather patterns. Extreme weather conditions in the areas in which our retail stores, suppliers, manufacturers, 
customers, distribution centers, offices, headquarters and vendors are located could adversely affect our operating results and 
financial condition. Moreover, natural disasters such as earthquakes, hurricanes, wildfires, tsunamis, floods or droughts, whether 
occurring in the United States or abroad, and their related consequences and effects, including energy shortages and public 
health issues, have in the past temporarily disrupted, and could in the future disrupt, ou